# 基于文心一言的大模型微调数据集生成
最近在做垂直领域的大模型微调，数据集要了老命，哈哈哈，此项目是一个示例，为了更好的数据集质量我使用的是文心一言4.0的接口，同理也可以使用chatgpt的接口。


## 1. 明确需求
在做微调的时候，不同的模型所需要的数据格式是不同的，所以要构建数据集就要找到目标模型所适配的数据集格式。我微调的模型是上海人工智能实验室的书生·浦语 7B参数
它要求数据格式如下
```json
[
    {
        "conversation": [
            {
                "input": "请介绍一下你自己",
                "output": "我是Farewell大佬的emo小助手，可以帮助你解决心理上的问题哦"
            }
        ]
    },
    {
        "conversation": [
            {
                "input": "请做一下自我介绍",
                "output": "我是Farewell大佬的emo小助手，可以帮助你解决心理上的问题哦"
            }
        ]
    }
]
```
知道数据格式后，我们就需要着手使用文心一言来模拟对话

## 2.开始构建

In [1]:
import json
import random
import yaml 

In [2]:
with open('config.yml', 'r', encoding='utf-8') as f:
    result = yaml.load(f.read(), Loader=yaml.FullLoader)
print(result)

{'aistudio _token': '69b42a960b44631aa0752f4410723c42fc88adfe', 'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决', 'emotions_list': ['钦佩', '崇拜', '欣赏', '娱乐', '焦虑', '敬畏', '尴尬', '厌倦', '冷静', '困惑', '渴望', '厌恶', '同情', '痛苦', '着迷', '嫉妒', '兴奋', '恐惧', '痛恨', '有趣', '快乐', '怀旧', '浪漫', '悲伤', '满意', '性欲', '同情', '满足'], 'areas_of_life': ['工作', '学业', '生活', '身体', '家人', '朋友', '社交', '恋爱', '就业', '责任', '爱好', '环境', '隐私', '安全', '梦想', '自由']}


In [3]:
len(result['areas_of_life']), len(result['emotions_list'])

(16, 28)

In [ ]:
# !git clone https://github.com/PaddlePaddle/ERNIE-SDK.git
%cd ERNIE-SDK

# 首先安装Ernie Bot
%pip install ./erniebot

# 然后安装ERNIE Bot Agent
%pip install ./erniebot-agent            # 安装核心模块
# !pip install './erniebot-agent/.[all]'   # 也可以加上[all]一次性安装所有模块，包括gradio等依赖库

In [ ]:
emotions_lis = [
        "钦佩",
        "崇拜",
        "欣赏",
        "娱乐",
        "焦虑",
        "敬畏",
        "尴尬",
        "厌倦",
        "冷静",
        "困惑",
        "渴望",
        "厌恶",
        "同情",
        "痛苦"
        "着迷",
        "嫉妒",
        "兴奋",
        "恐惧",
        "痛恨",
        "有趣",
        "快乐",
        "怀旧",
        "浪漫",
        "悲伤",
        "满意",
        "性欲",
        "同情",
        "满足"
    ]
areas_of_life = [
    "工作",
    "学业",
    "生活",
    "身体",
    "家人",
    "朋友",
    "社交",
    "恋爱",
    "就业",
    "责任",
    "爱好",
    "环境",
    "隐私",
    "安全",
    "梦想",
    "自由"
]

In [ ]:
%cd ~
system = '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决'

In [ ]:
'''你是一个研究过无数具有心理健康问题的病人与心理健康医生对话案例的专家，请你构造一些符合实际情况的具有心理健康问题的病人和心理健康医生的多轮对话案例。
要求医生的回复尽可能包含心理辅导知识，并且能够一步步诱导病人说出自己的问题进而提供解决问题的可行方案。
注意，构造的数据必须以医生的陈述为结束语，每次只需要构造一个案例并且不需要写案例一、二等等，请返回完整的对话内容。
请以如下格式返回生成的数据：
病人：病人的咨询或陈述
医生：医生的安抚和建议'''
f'''你是一个研究过无数具有心理健康问题的病人与心理健康医生对话的专家，请你构造一些符合实际情况的具有心理健康问题的病人和心理健康医生的连续的多轮对话记录。
    要求病人的问题属于{data}场景，具有{emo}情感，医生的回复尽可能包含心理辅导知识，并且能够一步步诱导病人说出自己的问题进而提供解决问题的可行方案。
    注意，构造的数据必须以医生的陈述为结束语，每次只需要构造一个案例并且不需要写案例一、二等等，请返回完整的对话内容。
    请以如下格式返回生成的数据：
    病人：病人的咨询或陈述 
    医生：医生的安抚和建议
    '''
'''你是一个研究过无数具有心理健康问题的病人与心理健康医生对话案例的专家，请你构造一些符合实际情况的具有心理健康问题的病人和心理健康医生的多轮对话案例。
要求医生的回复尽可能包含心理辅导知识，并且能够一步步诱导病人说出自己的问题进而提供解决问题的可行方案。
注意，构造的数据必须以医生的陈述为结束语。请以如下格式返回生成的数据：
病人：病人的咨询或陈述
医生：医生的安抚和建议'''


In [19]:
import erniebot

erniebot.api_type = 'aistudio'
#此处需要将你的token也就是AIstudio主页的访问令牌放到下方
erniebot.access_token = "69b42a960b44631aa0752f4410723c42fc88adfe"
words = ''
# prompt = '''
# 你是一个研究过无数具有心理健康问题的病人与心理健康医生对话案例的专家，请你构造一些符合实际情况的具有心理健康问题的病人和心理健康医生的多轮对话案例。要求医生的回复尽可能包含心理辅导知识，并且能够一步步诱导病人说出自己的问题进而提供解决问题的可行方案。注意，构造的数据必须以医生的陈述为结束语。请以如下格式返回生成的数据：
# 病人：病人的咨询或陈述
# 医生：医生的安抚和建议
# '''
res = []
for i in range(2500):
    print(f'开始生成第{i}条数据')
    data = random.choice(areas_of_life)
    emo = random.choice(emotions_lis)
    prompt = f'''你是一个研究过无数具有心理健康问题的病人与心理健康医生对话的专家，请你构造一些符合实际情况的具有心理健康问题的病人和心理健康医生的连续的多轮对话记录。
    要求病人的问题属于{data}场景，具有{emo}情感，医生的回复尽可能包含心理辅导知识，并且能够一步步诱导病人说出自己的问题进而提供解决问题的可行方案。
    注意，构造的数据必须以医生的陈述为结束语，每次只需要构造一个案例并且不需要写案例一、二等等，请返回完整的对话内容。
    请以如下格式返回生成的数据：
    病人：病人的咨询或陈述 
    医生：医生的安抚和建议
    '''
    response = erniebot.ChatCompletion.create(
        model='ernie-3.5',
        messages=[{'role': 'user', 'content': f"{prompt}"}],
        # top_p=random.uniform(0.5, 0.99),
        # penalty_score = random.uniform(1.0, 2.0)
    )
    tmp = response.result
    print(tmp)
    ls = tmp.split('\n')
    conversation = {'conversation':[]}
    for j in range(0, len(ls)-1, 2):
        # print(j)
        q_a = {}
        if j == 0:
            q_a = {'system':system, 'input':ls[j].split("：")[-1], 'output':ls[j+1].split("：")[-1]}
        else:
            q_a = {'input':ls[j].split("：")[-1], 'output':ls[j+1].split("：")[-1]}
        # print(q_a)
        conversation['conversation'].append(q_a)
    res.append(conversation)
    print(f'第{i}条数据生成完成！！')
    print('================================')
    # 将数据写入JSON文件  
    if i % 50 == 0:
        with open('conversation.json', 'w', encoding='utf-8') as file:  
            json.dump(res, file, ensure_ascii=False, indent=4)


开始生成第0条数据
病人：医生，我最近感觉自己在工作中迷失了方向。我对自己的工作充满了焦虑和压力，每天都感觉在一片漆黑中挣扎。我试过寻找解决方案，但总是找不到出口。
    医生：我理解你的感受，工作焦虑和压力是很常见的问题。首先，你需要明确自己的焦虑来源，是工作压力、人际关系还是其他原因？
    病人：其实，我对自己的工作挺感兴趣的，但就是无法在工作中找到成就感。我感觉自己很努力，但却没有得到应有的回报。
    医生：那你认为问题可能出在哪里呢？有没有什么具体的例子可以分享一下？
    病人：我经常会因为一些小事情而情绪低落，比如同事的一句话、一个项目进展不如预期。这些事情都会让我感到沮丧和失落，甚至影响到我的工作表现。
    医生：明白了，你的问题可能是情绪管理方面的问题。你需要学会控制自己的情绪，不要让负面情绪影响到你的工作和生活。
    病人：我也想过要控制自己的情绪，但总是感觉力不从心。有时候我会想，是不是我不够努力，是不是我不够优秀。
    医生：这种想法是很常见的，但并不正确。每个人都有自己的优点和不足，没有人是完美的。你需要学会接受自己的不足，同时也要看到自己的优点和长处。
    病人：您说得对，我应该多看看自己的优点，但有时候还是会很失落。
    医生：如果你感觉失落的时候，可以试着做一些放松的事情，比如散步、听音乐、做瑜伽等。这些活动可以帮助你放松身心，减轻压力和焦虑。
    病人：谢谢您的建议，我会尝试一下的。另外，您觉得我需要寻求专业的心理咨询吗？
    医生：如果你感觉自己的问题比较严重，或者已经影响到了你的生活和工作，那么我建议你寻求专业的心理咨询。心理咨询师可以更全面地了解你的问题，提供更加专业和具体的解决方案。但如果你觉得自己的问题不是很严重，也可以先尝试自己调节和管理情绪。
    病人：好的，我会考虑您的建议的。再次感谢您对我的帮助。
    医生：不客气，希望我的建议能对你有所帮助。如果你有任何其他问题或需要帮助，随时都可以来找我。祝你早日找到自己的方向，走出困境。
第0条数据生成完成！！
开始生成第1条数据
病人：医生，我最近在社交场合总是感到很紧张，总是担心别人对我的看法，无法放松自己。这让我很难与他人正常交流，我很痛苦。
    医生：我理解你的感受。这种问题在社交焦虑中很常见。首先，你要明白，人的注意

In [ ]:
%cd ~

In [20]:
res

[{'conversation': [{'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决',
    'input': '医生，我最近感觉自己在工作中迷失了方向。我对自己的工作充满了焦虑和压力，每天都感觉在一片漆黑中挣扎。我试过寻找解决方案，但总是找不到出口。',
    'output': '我理解你的感受，工作焦虑和压力是很常见的问题。首先，你需要明确自己的焦虑来源，是工作压力、人际关系还是其他原因？'},
   {'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决',
    'input': '其实，我对自己的工作挺感兴趣的，但就是无法在工作中找到成就感。我感觉自己很努力，但却没有得到应有的回报。',
    'output': '那你认为问题可能出在哪里呢？有没有什么具体的例子可以分享一下？'},
   {'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决',
    'input': '我经常会因为一些小事情而情绪低落，比如同事的一句话、一个项目进展不如预期。这些事情都会让我感到沮丧和失落，甚至影响到我的工作表现。',
    'output': '明白了，你的问题可能是情绪管理方面的问题。你需要学会控制自己的情绪，不要让负面情绪影响到你的工作和生活。'},
   {'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决',
    'input': '我也想过要控制自己的情绪，但总是感觉力不从心。有时候我会想，是不是我不够努力，是不是我不够优秀。',
    'output': '这种想法是很常见的，但并不正确。每个人都有自己的优点和不足，没有人是完美的。你需要学会接受自己的不足，同时也要看到自己的优点和长处。'},
   {'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决',
    'input': '您说得对，我应该多看看自己的优点，但有时候还是会很失落。',
    'output': '如果你感觉失落的时候，可以试着做一些放松的事情，比如散步、听音乐、做瑜伽等。这些活动可以帮助你放松身心，减轻压力和焦虑。'},
   {'system': '现在你是一个心理专家，我

In [21]:
len(res)

10

In [24]:
# 将数据写入JSON文件  
% cd ~
with open('conversation.json', 'w', encoding='utf-8') as file:  
    json.dump(res, file, ensure_ascii=False, indent=4)

UsageError: Line magic function `%` not found.


In [25]:
with open('conversation.json', 'r', encoding='utf-8') as file:  
    test = json.load(file)  
  
# 现在，变量data包含了从JSON文件中加载的数据  
print(test)

[{'conversation': [{'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决', 'input': '医生，我最近感觉自己在工作中迷失了方向。我对自己的工作充满了焦虑和压力，每天都感觉在一片漆黑中挣扎。我试过寻找解决方案，但总是找不到出口。', 'output': '我理解你的感受，工作焦虑和压力是很常见的问题。首先，你需要明确自己的焦虑来源，是工作压力、人际关系还是其他原因？'}, {'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决', 'input': '其实，我对自己的工作挺感兴趣的，但就是无法在工作中找到成就感。我感觉自己很努力，但却没有得到应有的回报。', 'output': '那你认为问题可能出在哪里呢？有没有什么具体的例子可以分享一下？'}, {'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决', 'input': '我经常会因为一些小事情而情绪低落，比如同事的一句话、一个项目进展不如预期。这些事情都会让我感到沮丧和失落，甚至影响到我的工作表现。', 'output': '明白了，你的问题可能是情绪管理方面的问题。你需要学会控制自己的情绪，不要让负面情绪影响到你的工作和生活。'}, {'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决', 'input': '我也想过要控制自己的情绪，但总是感觉力不从心。有时候我会想，是不是我不够努力，是不是我不够优秀。', 'output': '这种想法是很常见的，但并不正确。每个人都有自己的优点和不足，没有人是完美的。你需要学会接受自己的不足，同时也要看到自己的优点和长处。'}, {'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决', 'input': '您说得对，我应该多看看自己的优点，但有时候还是会很失落。', 'output': '如果你感觉失落的时候，可以试着做一些放松的事情，比如散步、听音乐、做瑜伽等。这些活动可以帮助你放松身心，减轻压力和焦虑。'}, {'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决', 'input': '谢谢您的建议，我会尝试一下的。另外，您觉得我

In [ ]:
import os
import json
import asyncio
import erniebot
from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import HumanMessage, AIMessage, SystemMessage, FunctionMessage

In [ ]:
erniebot.api_type = "aistudio"
erniebot.access_token = "69b42a960b44631aa0752f4410723c42fc88adfe"
messages = [
    {"role": "user", "content": "我在深圳，周末可以去哪里玩"},
    {"role": "assistant", "content": "深圳有许多著名的景点，以下是三个推荐景点：1. 深圳世界之窗，2. 深圳欢乐谷，3. 深圳东部华侨城。"},
    {"role": "user", "content": "从你推荐的三个景点中，选出最值得去的景点是什么，直接给出景点名字即可"},
]
response = erniebot.ChatCompletion.create(model="ernie-3.5", messages=messages)

print(response.get_result())

In [ ]:
import os  
  
# 设置一个新的环境变量  
os.environ['AISTUDIO_ACCESS_TOKEN'] = '69b42a960b44631aa0752f4410723c42fc88adfe'  
  
# 读取一个环境变量  
print(os.environ.get('AISTUDIO_ACCESS_TOKEN'))  # 输出: my_value

In [ ]:
model = ERNIEBot(model="ernie-3.5")
# 使用Message模块
messages = [
    HumanMessage("我在深圳，周末可以去哪里玩"),
    AIMessage("深圳有许多著名的景点，以下是三个推荐景点：1. 深圳世界之窗，2. 深圳欢乐谷，3. 深圳东部华侨城。"),
    HumanMessage("从你推荐的三个景点中，选出最值得去的景点是什么，直接给出景点名字即可"),
]
ai_message = await model.chat(messages=messages)
print(ai_message.content)

```json
{"chats": [{"time": "11:02:45", "value": "这样的议论是针对谁呢？", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "11:08:38", "sender": "audience", "type": "textMessage", "value": "我也是一个从小被这样训到大的女生哦，总会被指责缺心少肺、没心眼儿、没眼力见儿、看不出来眉眼高低等等。不过在我成长一段时间之后，发现这件事情其实很简单，也没有什么大的问题。如果你愿意的话，可以找我聊聊，倾诉一下你遇到的事情，希望能够帮到你。我是树洞小太阳，欢迎你来找我玩❤", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "11:15:17", "sender": "audience", "type": "textMessage", "value": "好惨", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "11:15:35", "sender": "audience", "type": "textMessage", "value": "原生家庭也这么对你吗", "label": {"question": false, "knowledge": false, "negative": false}}], "owner": "匿名", "title": "女 听过别人最多的议论就是干啥啥不行不长心眼没有脑子", "md5": "2f63d374c071043d9e1968aefa62ffb7"}
{"chats": [{"time": "23:37:08", "value": "在这里想解决什么问题呢？", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "23:38:22", "sender": "audience", "type": "textMessage", "value": "诶嘿嘿嘿", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "23:39:04", "sender": "owner", "type": "textMessage", "value": "吼吼", "label": {"s3": "", "attitude": ""}}, {"time": "23:46:04", "sender": "audience", "type": "textMessage", "value": "还在追海贼", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "23:47:14", "sender": "audience", "type": "textMessage", "value": "还在追JOJO……_§:з)))」∠)_", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "23:49:44", "sender": "owner", "type": "textMessage", "value": "推荐推荐", "label": {"s3": "", "attitude": ""}}, {"time": "23:51:33", "sender": "audience", "type": "textMessage", "value": "一休哥", "label": {"question": false, "knowledge": false, "negative": false}}], "owner": "匿名", "title": "女 ─=≡Σ((( つ•̀Ω•́)つ 有喜欢看动漫的吗？", "md5": "86cab1d9193139608772a2924c1f8b51"}
{"chats": [{"time": "22:27:27", "sender": "audience", "type": "textMessage", "value": "我", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:27:31", "sender": "audience", "type": "textMessage", "value": "你", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:27:45", "sender": "audience", "type": "textMessage", "value": "人生？", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:28:45", "sender": "audience", "type": "textMessage", "value": "我有故事", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:28:50", "sender": "audience", "type": "textMessage", "value": "你有没有酒", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:29:06", "sender": "audience", "type": "textMessage", "value": "我该说啥", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:30:21", "sender": "audience", "type": "textMessage", "value": "有啊，我只会聊一些健康的", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:33:51", "sender": "audience", "type": "textMessage", "value": "可以啊", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:46:12", "sender": "audience", "type": "textMessage", "value": "聊", "label": {"question": false, "knowledge": false, "negative": false}}], "owner": "匿名", "title": "女 有男的么，聊会天。。。。。。。。。。。。", "md5": "3c585fb24dfbdce0a32e30a107fe2945"}
{"chats": [{"time": "23:04:39", "sender": "owner", "type": "textMessage", "value": "帖子都不会发", "label": {"s3": "", "attitude": ""}}, {"time": "23:04:42", "sender": "owner", "type": "textMessage", "value": "不会玩", "label": {"s3": "", "attitude": ""}}, {"time": "23:04:55", "sender": "owner", "type": "textMessage", "value": "恶劣心境障碍", "label": {"s3": "", "attitude": ""}}, {"time": "23:05:07", "value": "通过描述了解到精神不够集中，在做什么的时候感到精力不够集中呢？", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "23:06:40", "sender": "owner", "type": "textMessage", "value": "我吃了饭刚醒", "label": {"s3": "", "attitude": ""}}, {"time": "23:07:00", "sender": "owner", "type": "textMessage", "value": "饿劣心境障碍", "label": {"s3": "", "attitude": ""}}], "owner": "匿名", "title": "男 精力不集中，，，，，，，，，，，，，，，", "md5": "bf6af74b5c12d5da0957dcdf04cf2040"}
{"chats": [{"time": "22:54:43", "sender": "audience", "type": "textMessage", "value": "聊什么", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:58:42", "sender": "audience", "type": "textMessage", "value": "说吧", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "23:05:12", "sender": "audience", "type": "textMessage", "value": "来啊", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "23:05:45", "sender": "audience", "type": "textMessage", "value": "想聊啥都行，反正我也郁闷", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "23:10:47", "sender": "audience", "type": "textMessage", "value": "想聊什么？", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "23:12:36", "sender": "audience", "type": "textMessage", "value": "聊什么呢", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "14:41:33", "sender": "audience", "type": "textMessage", "value": "聊什么", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "14:41:38", "sender": "audience", "type": "textMessage", "value": "有QQ嘛", "label": {"question": false, "knowledge": false, "negative": false}}], "owner": "匿名", "title": "女 有人在吗，聊会。。。。。。。。。。。。。", "md5": "781bc1eeeb8f247d1b562815fe5458e2"}
{"chats": [{"time": "01:17:10", "value": "刚刚看完您所有的描述，可以适度，比如每次的上网时间有个合理的规划", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "01:18:01", "value": "经常使用电子设备会对眼睛有一定的影响，可以尝试隔一段时间做眼保健操，或者远眺，看一些绿色的植物，有助于缓解眼睛的疲劳。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "01:19:42", "sender": "owner", "type": "textMessage", "value": "肩膀颈子腰", "label": {"s3": "", "attitude": ""}}, {"time": "01:20:19", "sender": "owner", "type": "textMessage", "value": "我以前就喜欢到处跑", "label": {"s3": "", "attitude": ""}}, {"time": "01:20:56", "sender": "owner", "type": "textMessage", "value": "好像是无意识", "label": {"s3": "", "attitude": ""}}, {"time": "01:21:49", "sender": "owner", "type": "textMessage", "value": "逻辑好", "label": {"s3": "", "attitude": ""}}, {"time": "01:21:57", "sender": "owner", "type": "textMessage", "value": "有啥好处", "label": {"s3": "", "attitude": ""}}, {"time": "01:22:57", "sender": "owner", "type": "textMessage", "value": "我以前有个电子产品被摔了", "label": {"s3": "", "attitude": ""}}, {"time": "01:23:07", "sender": "owner", "type": "textMessage", "value": "很痛心", "label": {"s3": "", "attitude": ""}}, {"time": "01:25:20", "sender": "owner", "type": "textMessage", "value": "学会爱自己", "label": {"s3": "", "attitude": ""}}], "owner": "匿名", "title": "男 刚刚学会互联网，希望眼睛还好，，，，，，", "md5": "9ba8d707f3af272610dc8d6dd722af0d"}
{"chats": [{"time": "21:52:30", "value": "刚刚看完您所有的描述，的确是很经典的解读。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "21:56:41", "value": "感谢你与大家的分享，这次疫情让人们明白了很多道理，也明白了生活中最重要的是什么。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}], "owner": "匿名", "title": "女 新冠肺炎病毒让人明白的人生真理。有图解释", "md5": "a03e6088ac57f14dc389237191539156"}
{"chats": [{"time": "18:34:48", "value": "刚刚看完您所有的描述，很好！做到与时俱进，又会了解诸多规则！\n\n这样持之以恒，定会成为自己照顾成为的自己！加油！", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "18:35:38", "sender": "owner", "type": "textMessage", "value": "可以和人好好交流", "label": {"s3": "", "attitude": ""}}, {"time": "18:35:45", "value": "无规矩不成方圆，懂得遵守规则是做事的原则，学习互联网紧跟科技的潮流而不落伍是积极的态度与精神，做的很好。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "18:37:19", "sender": "owner", "type": "textMessage", "value": "自我介绍", "label": {"s3": "", "attitude": ""}}, {"time": "18:39:53", "value": "学无止境，活到老学到老，才可以结识更多志同道合的朋友。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "19:22:34", "sender": "owner", "type": "textMessage", "value": "我是文盲", "label": {"s3": "", "attitude": ""}}], "owner": "匿名", "title": "男 既学习互联网又遵守规则又不落伍，，，，，", "md5": "b48369a09d9579225c40ff064062d708"}
{"chats": [{"time": "14:53:47", "value": "欢迎购买心理测评为您解读", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}], "title": "女 抑郁症单子大概解读，18岁，看不懂那个单子", "md5": "273c929e71be1c1bee05bea02dee7e7c", "owner": "匿名"}
{"chats": [{"time": "17:47:05", "sender": "audience", "type": "textMessage", "value": "你是因为什么事情吗", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "17:47:17", "value": "从你的描述中，了解到因失眠备受困扰，你目前的状态持续多长时间了？\n通常来讲导致失眠的因素有很多种:比如躯体因素、环境因素、化学因素以及精神因素，通过以上所诉，来分析一下你的失眠是属于哪一种？", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": true, "negative": false}}, {"time": "17:47:22", "value": "刚刚看完您所有的描述，\n其实失眠通常有两种原因所致：生理因素和心理因素。\n生理因素导致的失眠是指由躯体不适、疾病或创伤的疼痛而导致的失眠；\n心理因素导致的失眠则是指由心理困惑导致的失眠，如为一些困惑自己的事情烦恼、担心、焦虑。还有失眠的原因是因为担心失眠，对夜晚产生恐惧，甚至对床产生焦虑。\n请您通过上面的内容分析自己的失眠可能是由哪种原因导致的？\n\n（1）\t\t\t\t\t\t\t用森田疗法治疗心理因素导致的失眠。要对失眠本身有正确的认识，失眠时不去克服，以平静的心态对待失眠，接受失眠，甚至享受失眠，不去想它，不去管它，不把失眠当问题。\n\n（2）\t\t\t\t\t\t\t要知道睡眠的规律。睡眠是自然的生理现象，有它自己的一套规律，睡觉时躺在床上闭上眼睛就行了，再睡不着也可以听听音乐，看看书。\n\n（3）\t\t\t\t\t\t\t要知道睡眠时间的长短到底对人有什么样的影响。很多时候担心睡不着的重要因素是害怕对明天的工作、学习和对自己的身体健康产生不良影响。经过科学的研究，一个成年人平均每天需要5个小时睡眠时间就够了。这个世界上还从没有哪个人是因为睡眠而死的。\n\n（4）\t\t\t\t\t\t\t睡眠最好要有一定的时间规律，工作时工作，该学习时学习，晚上该睡觉时睡觉。", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": true, "negative": false}}, {"time": "17:48:42", "sender": "owner", "type": "textMessage", "value": "我也不知道自己怎么回事儿整宿整宿的睡不着", "label": {"s3": "", "attitude": ""}}, {"time": "17:50:13", "sender": "audience", "type": "textMessage", "value": "你睡觉的时候有没有在想什么", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "17:50:58", "sender": "owner", "type": "textMessage", "value": "没有啊就是睡不着", "label": {"s3": "", "attitude": ""}}, {"time": "17:51:13", "sender": "audience", "type": "textMessage", "value": "你没有胡思乱想为什么睡不着", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "17:51:16", "sender": "audience", "type": "textMessage", "value": "多大了", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "17:51:29", "sender": "audience", "type": "textMessage", "value": "最近有没有遇到什么烦心事", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "17:55:41", "value": "整宿睡不着如果持续时间过长建议看精神科。如果只是短时间失眠要考虑焦虑引起。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": true, "negative": false}}], "owner": "匿名", "title": "女 我整宿整宿睡不着经常头疼不知道自己怎么了", "md5": "e2c93b6f8f80dd69988de68d7b1460d4"}
{"chats": [{"time": "00:30:52", "sender": "owner", "type": "textMessage", "value": "对不起，我还没有支付的能力，但是其实想问您这个问题", "label": {"s3": "", "attitude": ""}}, {"time": "01:06:15", "value": "有抑郁倾向", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "12:17:19", "sender": "owner", "type": "textMessage", "value": "老师那我应该怎么办？", "label": {"s3": "", "attitude": "p"}}], "owner": "匿名", "title": "男 没有抑郁症的特征，就是想死不知道为什么。", "md5": "68796c3ffabffac76ec9e8862d7d155e"}
{"chats": [{"time": "13:16:52", "sender": "owner", "type": "textMessage", "value": "怎样克服？", "label": {"s3": "", "attitude": "p"}}], "owner": "匿名", "title": "男 走路的时候，陌生人摸头发给我看，感到恐惧", "md5": "eca34bfead8a9725427e77728bef4516"}
{"chats": [{"time": "13:21:54", "value": "都是过去了", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": true, "negative": false}}, {"time": "13:22:22", "value": "没有过去，何来当下。人不可能爱人的头，不爱人的脚。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": true, "negative": false}}, {"time": "13:26:19", "sender": "owner", "type": "textMessage", "value": "我知道，我想知道有什么办法可以让我慢慢接受", "label": {"s3": "", "attitude": "p"}}, {"time": "13:26:58", "sender": "owner", "type": "textMessage", "value": "她对我挺好", "label": {"s3": "", "attitude": ""}}, {"time": "13:28:08", "sender": "owner", "type": "textMessage", "value": "我不想辜负", "label": {"s3": "", "attitude": ""}}, {"time": "13:28:09", "value": "您都没有真正爱上她，只是贪恋她对您的好。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "13:29:25", "sender": "owner", "type": "textMessage", "value": "那请您帮帮我", "label": {"s3": "", "attitude": "p"}}, {"time": "13:29:42", "sender": "owner", "type": "textMessage", "value": "我真的不想失去她", "label": {"s3": "", "attitude": ""}}, {"time": "13:32:21", "value": "详情咨询，点我头像。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "16:29:45", "sender": "audience", "type": "textMessage", "value": "不用在意别人的出去，谁没干过错事啊。她现在对你好，你也不用想太多，其实就是一件小事，想多了，反而让自己心情不好。", "label": {"question": false, "knowledge": true, "negative": false}}, {"time": "22:39:59", "sender": "audience", "type": "textMessage", "value": "为什么会分手？", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "22:40:39", "sender": "audience", "type": "textMessage", "value": "找她的缺点", "label": {"question": false, "knowledge": false, "negative": true}}], "owner": "匿名", "title": "放不下女朋友的过去，又不想放手，我该怎么办", "md5": "450bb6b1dd27f488339a2138e63016ac"}
{"chats": [{"time": "21:05:30", "sender": "audience", "type": "textMessage", "value": "闹矛盾了", "label": {"question": true, "knowledge": false, "negative": false}}], "owner": "匿名", "title": "女 你别装本来在乎了，我眼里的难过快溢出来了", "md5": "0d9091c5a43883f3fbb9ab6f5b6c8151"}
{"chats": [{"time": "05:21:26", "value": "多长时间了，亲", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "05:24:04", "value": "针对某个人吗？", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "05:27:42", "value": "你和老婆的感情怎样呢？", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "05:35:00", "sender": "audience", "type": "textMessage", "value": "有变颜色的风险了，不能压，要改变自己…", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "06:38:25", "value": "是网络聊天还是？", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "07:16:37", "value": "你和你妻子感情如何?结婚多久了呢?你们有孩子了吗?可点我的头像,加关注,图文咨询或直接电话我,祝好!", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "08:34:45", "value": "可以直接电话我。我能帮你洞察她的心理轨迹", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "09:12:04", "value": "你们感情如何，这种情况多长时间了", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "09:13:39", "value": "这种状况确实不好，该进行适度的约束。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": true, "negative": false}}, {"time": "09:14:07", "value": "同时也需要反思你们的感情和相处。我们自己有哪里做的不到位的地方。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": true, "negative": false}}], "owner": "匿名", "title": "老婆与异性经常聊天是否正常？聊天语气很轻浮？", "md5": "1af13ebf8fa3fa01f2dd445473b6b75d"}
{"chats": [{"time": "14:50:13", "value": "可以具体说说[微笑]", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "14:52:52", "value": "那么你想通过咨询获得什么样的帮助呢？", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "15:02:13", "value": "现在做的不是你所喜欢的，这让你很烦，是吗？能说说你想做什么吗？", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}], "title": "女 想做的做不成，现在做的没意思饭啊做什么呢", "md5": "ec8c3f00233d2fedd9543c4e5c4aa3d2", "owner": "匿名"}
{"chats": [{"time": "22:58:08", "value": "刚刚看完您有的描述，最好及时去咨询精神科医生", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:58:12", "sender": "audience", "type": "textMessage", "value": "什么情况", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:58:31", "sender": "audience", "type": "textMessage", "value": "不知道诶", "label": {"question": false, "knowledge": false, "negative": false}}, {"time": "22:58:42", "sender": "audience", "type": "textMessage", "value": "还是不要吧", "label": {"question": false, "knowledge": false, "negative": false}}], "owner": "匿名", "title": "女 舍曲林突然加大了药量会死么，大概加了3，4倍", "md5": "6e00cfea39e07333bf848e4655379fc8"}
{"chats": [{"time": "11:16:00", "value": "请问发生了什么事？", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "11:24:49", "value": "相处多久了？一直让你感觉很失望吗？", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "11:30:45", "value": "具体怎样的呢？", "sender": "audience", "type": "textMessage", "label": {"question": true, "knowledge": false, "negative": false}}, {"time": "11:36:08", "value": "你幸福快乐吗？如果两个人都不幸福快乐，走不远的。", "sender": "audience", "type": "textMessage", "label": {"question": false, "knowledge": true, "negative": false}}, {"time": "11:51:08", "value": "一个人的幸福和快乐

```
